# S&P500 trend follwing on mt5 

Optimizing the indicators to find the best blend



### Result



In [1]:
import MetaTrader5 as mt5
import pandas as pd
import ta
import time
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from datetime import datetime
mt5.initialize()

True

In [2]:
symbol = 'US500'
timeframe = mt5.TIMEFRAME_M15
start_date = datetime(2023,1,1)
end_time = datetime.now()

def get_data(symbol, timeframe, start_date, end_time):
    rates = mt5.copy_rates_range(symbol, timeframe, start_date, end_time)
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('time', inplace=True)
    df.columns = df.columns.str.capitalize()
    return df

us500 = get_data(symbol, timeframe, start_date, end_time)
us500.dropna(inplace=True)


In [53]:
class BollingerBandBuy(Strategy):
    initsize = 5.0
    mysize = initsize
    bb_window = 25
    
    def init(self):
        super().init()
        self.lower_bb = self.I(ta.volatility.bollinger_lband, pd.Series(self.data.Close), window=self.bb_window)
        
        
    def next(self):
        super().next()
            
        if (self.data.Close[-1] > self.lower_bb[-1]) and (self.data.Close[-2] <= self.lower_bb[-2]):
            sl1 = self.data.Close[-1] - 250e-1
            tp1 = self.data.Close[-1] + 550e-1
            self.buy(sl=sl1, tp=tp1, size=self.mysize)      
            
# Assuming us500 is defined elsewhere
bt = Backtest(us500, BollingerBandBuy, cash=10000, exclusive_orders=True, margin=1/100)

output = bt.run()
output


Start                     2023-01-03 01:00:00
End                       2024-03-28 23:45:00
Duration                    450 days 22:45:00
Exposure Time [%]                   83.887617
Equity Final [$]                      15389.3
Equity Peak [$]                       15464.7
Return [%]                             53.893
Buy & Hold Return [%]               36.215101
Return (Ann.) [%]                   41.402744
Volatility (Ann.) [%]               30.191716
Sharpe Ratio                         1.371328
Sortino Ratio                        2.995462
Calmar Ratio                         2.555321
Max. Drawdown [%]                  -16.202562
Avg. Drawdown [%]                   -0.616159
Max. Drawdown Duration      120 days 07:00:00
Avg. Drawdown Duration        1 days 14:23:00
# Trades                                  818
Win Rate [%]                        34.841076
Best Trade [%]                        1.42835
Worst Trade [%]                     -0.667623
Avg. Trade [%]                    

In [47]:


# Run optimization
optimization_results = bt.optimize(bb_window=range(5, 100, 5), maximize='Return [%]')

optimization_results

  0%|          | 0/10 [00:00<?, ?it/s]

Start                     2023-01-03 01:00:00
End                       2024-03-28 23:45:00
Duration                    450 days 22:45:00
Exposure Time [%]                   83.887617
Equity Final [$]                      15389.3
Equity Peak [$]                       15464.7
Return [%]                             53.893
Buy & Hold Return [%]               36.215101
Return (Ann.) [%]                   41.402744
Volatility (Ann.) [%]               30.191716
Sharpe Ratio                         1.371328
Sortino Ratio                        2.995462
Calmar Ratio                         2.555321
Max. Drawdown [%]                  -16.202562
Avg. Drawdown [%]                   -0.616159
Max. Drawdown Duration      120 days 07:00:00
Avg. Drawdown Duration        1 days 14:23:00
# Trades                                  818
Win Rate [%]                        34.841076
Best Trade [%]                        1.42835
Worst Trade [%]                     -0.667623
Avg. Trade [%]                    